In [2]:
# Importar bibliotecas
import pandas as pd
import geopandas as gp


# Introdução ao Problema
O objetivo deste algoritmo é gerar um conjunto de pares de endereços na cidade de São Paulo para o treinamento de um modelo de precificação de corridas de Uber.

O algoritmo terá natureza estocástica, dando preferência para endereços de chegada em distritos com maior densidade populacional e renda média por habitante (fatores relevantes para o uso do aplicativo Uber).

Para os endereços de saída, o algoritmo usará uma função de decaimento exponencial $P(d)$ (descrita abaixo) para gerar uma distância máxima para o endereço do destino, onde escolherá o endereço de destino uniformemente dentro desse intervalo.

Usaremos o conjunto de dados geográficos (disponível como um arquivo `.shape` com um grafo representando as ruas de São Paulo) distribuído pela plataforma [GeoSampa](https://geosampa.prefeitura.sp.gov.br) da Prefeitura de São Paulo para acessar os endereços disponíveis dentro da distância $d$ da origem.

# Implementação em Código Python

A implementação segue as etapas principais descritas abaixo:

1. __Carregamento e Preparação dos Dados:__

   - Utilize bibliotecas como `geopandas` para manipular arquivos `.shapefile`.
   - Construa o grafo das ruas de São Paulo com `networkx`.
2. __Cálculo das Probabilidades de Escolha:__

   - Importe os dados do IBGE utilizando `pandas`.
   - Calcule e normalize as probabilidades com base na população e renda dos distritos.
3. __Seleção do Endereço de Origem:__

   - Escolha aleatória de uma rua com `numpy.random.choice`.
   - Geração de um número dentro do intervalo válido.
4. __Seleção do Endereço de Destino:__

   - Use $P(d)$ para calcular o raio máximo:

     $$
     P(d) = \frac{1}{\sqrt{2\pi}\sigma} e^{-\frac{d^2}{2\sigma^2}}
     $$
   - Escolha uniformemente dentro do raio usando coordenadas do grafo.
5. **Visualização dos Resultados:**

   - Use `matplotlib` ou `folium` para criar mapas interativos.

---

# Visualização e Validação

Após a implementação, as corridas simuladas podem ser visualizadas em mapas interativos:

- **Mapas com `folium`:** Representam pares de origem e destino.
- **Gráficos:** Analisam estatisticamente a distribuição das distâncias e destinos.

Além disso, as corridas geradas podem ser exportadas para arquivos `.csv` para uso em modelos de treinamento.

In [1]:
# Carregar os dados
df = pd.read_csv('data/population_parameters_rates_by_district.csv')
df = df[['Cod_distrito', 'Nome_distrito', 'Pop_2010', 'Pop_2020', 'Pop/ha_2010']]
df_renda = pd.read_csv('data/renda_media_distrito.csv')

# Remover espaços em branco e converter para minúsculas
df['Nome_distrito'] = df['Nome_distrito'].str.strip().str.lower()
df_renda['Nome_distrito'] = df_renda['Nome_distrito'].str.strip().str.lower()

# Verificar valores ausentes
print(df['Nome_distrito'].isnull().sum())
print(df_renda['Nome_distrito'].isnull().sum())

# Definir intervalos de renda e seus pontos médios
income_intervals = {
    'Menos de 2 SM': 1,
    'De 2 a Menos de 5 SM': 3.5,
    'De 5 a Menos de 10 SM': 7.5,
    'De 10 a Menos de 15 SM': 12.5,
    'De 15 a Menos de 25 SM': 20,
    'De 25 SM e Mais ': 30
}

# Calcular a renda média para cada distrito
for col, midpoint in income_intervals.items():
    df_renda[col] = df_renda[col]/100 * midpoint

df_renda['average_income'] = df_renda[list(income_intervals.keys())].sum(axis=1)

# Normalizar a renda média
min_income = df_renda['average_income'].min()
max_income = df_renda['average_income'].max()
df_renda['normalized_income'] = (df_renda['average_income'] - min_income) / (max_income - min_income)

# Normalizar a população
min_pop = df['Pop_2020'].min()
max_pop = df['Pop_2020'].max()
df['normalized_pop'] = (df['Pop_2020'] - min_pop) / (max_pop - min_pop)

# Mesclar os dois dataframes na coluna 'Nome_distrito'
result = pd.merge(df, df_renda, on='Nome_distrito')

# Calcular a pontuação composta (média simples das normalizações)
result['composite_score'] = (result['normalized_income'] + result['normalized_pop']) / 2

# Normalizar a pontuação composta para obter probabilidades
total_score = result['composite_score'].sum()
result['probability'] = result['composite_score'] / total_score

# Exibir o resultado final com as probabilidades
print(result[['Nome_distrito', 'average_income', 'Pop_2020', 'normalized_income', 'normalized_pop', 'composite_score', 'probability']].head())

0
0


KeyError: 'menos de 2 SM'

In [15]:
origin = result[['Nome_distrito','Pop/ha_2010', 'average_income']]
origin['Pop/ha_2010_normalized'] = origin['Pop/ha_2010'] / origin['Pop/ha_2010'].sum()
origin['average_income_normalized'] = origin['average_income'] / origin['average_income'].sum()
origin['probability'] = (origin['Pop/ha_2010_normalized'] + origin['average_income_normalized']) / 2

,Nome_distrito,Pop/ha_2010,average_income,Pop/ha_2010_normalized,average_income_normalized,probabilty
0,agua rasa,123.13,12.96140,0.011757,0.011606,0.011681
1,alto de pinheiros,56.00,21.59010,0.005347,0.019332,0.012339
2,anhanguera,19.78,6.30320,0.001889,0.005644,0.003766
3,aricanduva,135.79,9.86990,0.012965,0.008837,0.010901
4,artur alvim,159.50,9.23100,0.015229,0.008265,0.011747
...,...,...,...,...,...,...
90,vila medeiros,168.73,9.36830,0.016111,0.008388,0.012249
91,vila prudente,105.29,11.57800,0.010053,0.010367,0.010210
92,vila sonia,109.54,15.08430,0.010459,0.013506,0.011983
93,sao domingos,84.84,11.19020,0.008101,0.010020,0.009060
